In [5]:
# import chess.pgn

# pgn = open("lichess_db_standard_rated_2014-09.pgn")

# # Read just 1 game for testing
# first_game = chess.pgn.read_game(pgn)
# print(first_game.headers)
# print(first_game.mainline_moves())

In [ ]:
# Start by activating the virtual environment: source chess-nlp-env/bin/activate

import chess.pgn
import pandas as pd
import itertools

pgn_path = "../data/processed/lichess_db_standard_rated_2014-09.pgn"
output_csv = "../data/processed/chess_games.csv"

games_data = []
num_games = 5000  # First 5000 games from the dataset

with open(pgn_path, encoding="utf-8") as pgn_file:
    for i in range(num_games):
        game = chess.pgn.read_game(pgn_file)
        if game is None:
            break  # end of file

        try:
            white = game.headers.get("White", "?")
            black = game.headers.get("Black", "?")
            result = game.headers.get("Result", "?")
            date = game.headers.get("Date", "?")
            site = game.headers.get("Site", "?")

            board = game.board()
            moves = []
            for move in game.mainline_moves():
                try:
                    san = board.san(move)
                    moves.append(san)
                    board.push(move)
                except Exception:
                    # Skip illegal move
                    moves = []
                    break

            if not moves:
                continue  # skip this game

            games_data.append({
                "white": white,
                "black": black,
                "result": result,
                "date": date,
                "site": site,
                "moves": " ".join(moves)
            })
        except Exception as e:
            # Skip the whole game if anything else fails
            continue

df = pd.DataFrame(games_data)
df.to_csv(output_csv, index=False)

print(f"✅ Extracted {len(df)} valid games to {output_csv}")
print(df.head())

ModuleNotFoundError: No module named 'chess'

In [ ]:
# Preprocessing